Importando biblioteca

In [ ]:
import pandas as pd

Carregando dataset na variável data

In [ ]:
data = pd.read_csv("datasets\kc_house_data.csv")
data.head()


Criando a coluna com níveis

Utilizando loop for

In [ ]:
for i in range(len(data)):
    if (data.loc[i, 'price'] > 0) & (data.loc[i, 'price'] < 321950):
        data.loc[i, 'nivel'] = 'nivel_0'
    elif (data.loc[i, 'price'] >= 321950) & (data.loc[i, 'price'] < 450000):
        data.loc[i, 'nivel'] = 'nivel_1'
    elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
        data.loc[i, 'nivel'] = 'nivel_2'
    else:
        data.loc[i, 'nivel'] = 'nivel_3'


In [ ]:
data.head(30)

Respondendo as perguntas do CEO

In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim( user_agent='geoapiExercises')

response = geolocator.reverse( '47.5112,-122.257'  )

print(response.raw['address']['road'])
print(response.raw['address']['house_number'])
print(response.raw['address']['neighbourhood'])
print(response.raw['address']['city'])
print(response.raw['address']['county'])
print(response.raw['address']['state'])


In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

# Create empty rows
data['road'] = 'NA'
data['house_number'] = 'NA'
data['neighbourhood'] = 'NA'
data['city'] = 'NA'
data['county'] = 'NA'
data['state'] = 'NA'

geolocator = Nominatim(user_agent='geoapiExercises')



for i in range( len(data) ):
    print( f'Loop: {i}/{len(data)}')
    
    # Make query
    query = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
    
    # API Request
    response = geolocator.reverse(query)
    
    # Populate data
    if 'house_number' in response.raw['address']:
        data.loc[i, 'house_number'] = response.raw['address']['house_number']
        
    if 'road' in response.raw['address']:
        data.loc[i, 'road'] = response.raw['address']['road']
        
    if 'neighbourhood' in response.raw['address']:
        data.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']
        
    if 'city' in response.raw['address']:
        data.loc[i, 'city'] = response.raw['address']['city']
        
    if 'county' in response.raw['address']:
        data.loc[i, 'county'] = response.raw['address']['county']
        
    if 'state' in response.raw['address']:
        data.loc[i, 'state'] = response.raw['address']['state']





Filtros interativos no mapa

In [ ]:
import plotly.express as px

In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

houses = data[['id', 'lat', 'long', 'price']].copy()

# Define levels
for i in range(len( houses )):
    if houses.loc[i, 'price'] <= 321950:
        houses.loc[i, 'level'] = 0
        
    elif (houses.loc[i, 'price'] > 321950) & (houses.loc[i, 'price'] <= 450000):
        houses.loc[i, 'level'] = 1
        
    elif (houses.loc[i, 'price'] > 450000) & (houses.loc[i, 'price'] <= 645000):
        houses.loc[i, 'level'] = 2
        
    else:
        houses.loc[i, 'level'] = 3
        
houses['level'] = houses['level'].astype(int)

fig = px.scatter_mapbox( houses,
                        lat='lat',
                        lon='long',
                        color='level',
                        size='price',
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=15,
                        zoom=10)

fig.update_layout( mapbox_style='open-street-map')
fig.update_layout( height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
fig.show()


# Adicionando filtros iterativos

In [28]:
import ipywidgets as widgets
from ipywidgets import fixed

In [31]:
df = pd.read_csv('datasets\kc_house_data.csv')

df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no')

df['level'] = df['price'].apply(lambda x: 0 if x < 321950 else
                                        1 if (x > 321950) & ( x< 450000) else
                                        2 if (x > 450000) & (x < 645000) else 3)

df['level'] = df['level'].astype(int)
style = {'description_width': 'initial'}

# Iterative buttons
price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 77000000,
    step = 1,
    description='Maximun Price',
    disable=False,
    style = style
    
)

waterfront_bar = widgets.Dropdown(
    options = df['is_waterfront'].unique().tolist(),
    value='yes',
    description='Water View',
    disable=False
)

def update_map(df, waterfront, limit):
    houses = df[(df['price'] <= limit) & 
                (df['is_waterfront'] == waterfront)][['id', 'lat', 'long', 'price', 'level']]
    
    fig = px.scatter_mapbox(houses,
                            lat='lat',
                            lon='long',
                            color='level',
                            size='price',
                            color_continuous_scale=px.colors.cyclical.IceFire,
                            size_max=15,
                            zoom=10)

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
    fig.show()


In [34]:
widgets.interactive(update_map, df=fixed( df ), waterfront=waterfront_bar, limit=price_limit)

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

Iteratividade com Dashboard


In [36]:
import ipywidgets as widgets
from matplotlib import gridspec
from matplotlib import pyplot as plt

In [47]:
# Prepare dataset 
data = pd.read_csv('datasets\kc_house_data.csv')

# Change date format
data['year'] = pd.to_datetime( data['date']).dt.strftime( '%Y' )
data['date'] = pd.to_datetime( data['date']).dt.strftime( '%Y-%m-%d' )
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U' )

# Widgets to control data
date_limit = widgets.SelectionSlider(
    options=df['date'].sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Disponível',
    continuous_update = False,
    orientation = 'horizontal',
    readout=True)

def update_map( data, limit ):
    # Filter data
    df = data[data['date'] >= limit].copy()
    
    fig = plt.figure( figsize=(21,12))
    specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig )
    
    ax1 = fig.add_subplot( specs[0, :]) # First rows
    ax2 = fig.add_subplot( specs[1, 0]) # Second row First Column
    ax3 = fig.add_subplot( specs[1, 1]) # Second Row Second Column rows
    
    by_year = df[['id', 'year']].groupby( 'year' ).sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )
    
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    ax2.plot(by_day['date'], by_day['id'] )
    ax2.set_title( 'Avg Price by Day' )
    
    by_week_of_year = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    ax3.bar(by_week_of_year['year_week'], by_week_of_year['id'])
    ax3.set_title('Avg Price by Week of Year')
    plt.xticks( rotation=60 )
    
    


In [48]:
widgets.interactive( update_map, data=fixed(df), limit=date_limit )

interactive(children=(SelectionSlider(continuous_update=False, description='Disponível', index=212, options=('…